In [1]:
import pandas as pd

In [2]:
from openai import OpenAI

client = OpenAI()

In [4]:
df = ds.to_pandas()

In [5]:
df = df.drop_duplicates(subset=['Description', 'Patient'], )

In [6]:
df.to_csv('../data/data.csv', index=False)

In [7]:
df = pd.read_csv('../data/data.csv')

In [20]:
sampled_df = df.sample(n=1000, random_state=42)

In [26]:
# Continuing from the previous example
sampled_df = sampled_df.reset_index(drop=True)
sampled_df['id'] = sampled_df.index

In [27]:
sampled_df = sampled_df[['id'] + [col for col in sampled_df.columns if col != 'id']]

In [28]:
sampled_df['id'] = sampled_df.index + 1

In [29]:
sampled_df.head()

,id,Description,Patient,Doctor
0,1,Are there any side effects of misprostal 200mg...,"HI DR, ACTUALLY I HAVE MISSED MY FIRST PERIOD ...","Hello, Your pregnancy test positive.If you don..."
1,2,What causes pitting edema of the lower extremi...,i have ED and associated with pitting edema of...,"Hello,As you underwent surgery of that foot, s..."
2,3,Suggest treatment for black teeth in a 2 year ...,"Hi,my son is 2y/o n his 4 front teeth are blac...",Thanks for using Health care Magic.Read your q...
3,4,Why am I suffering from rashes and pimples in ...,I have a rash on my inner arms and inner forea...,"hi there.1. by your description, your concern ..."
4,5,Can a fetus be alive despite the absence of fe...,apparently im 7 weeks pregnant i went to get a...,Hello and Welcome to ‘Ask A Doctor’ service. I...


In [33]:
documents = sampled_df.to_dict(orient='records')

In [8]:
prompt_template = """
You emulate a user of our health care assistant application.
Formulate 5 questions this user might ask based on a provided patient question.
Make the questions specific to the description.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Description: {Description}
Patient: {Patient}
Doctor: {Doctor}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [14]:
prompt = prompt_template.format(**documents[2])

In [15]:
print(prompt)

You emulate a user of our health care assistant application.
Formulate 5 questions this user might ask based on a provided patient question.
Make the questions specific to the description.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Description: Q. I have started to get lots of acne on my face, particularly on my forehead. Please help me.
Patient: Hi doctor! I used to have clear skin but since I moved to a new place, I started to have lots of acne on my face particularly on my forehead. I thought it would disappear once I went back home, but it only got worse. I did some research and assumed that it was caused by drinking too much cow's milk, but it has been since since I stopped and they would still not go away. I also noticed that I get deep acne whenever I'm nearing my period, along with the usual small red bumps. I bought an acne soap and have been using

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
questions = llm(prompt)

In [19]:
print(questions)

{"questions": ["What factors could have contributed to the sudden onset of my acne after moving?", "Can you explain the role of cow's milk in causing acne and why stopping it hasn't helped?", "What specific oral and topical medications would you recommend for my type of acne?", "How can I determine the grade of my acne, and why is it important for treatment?", "Are there any additional skincare steps or products I should incorporate along with the acne soap?"]}


In [30]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [35]:
from tqdm.auto import tqdm
import json

In [32]:
results = {}

In [ ]:
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
final_results = []

for doc_id, questions in results.item():
    for q in questions:
        final_results.append((doc_id, q))

In [ ]:
final_results[0]

In [ ]:
df_results = pd.DataFrame(final_results, columns=['id', 'question'])

In [ ]:
df_results.to_csv('../data/ground-truth-retrieval.csv', index=False)